#Enterprise Fleet Analytics Pipeline: Focuses on the business outcome (analytics) and the domain (fleet/logistics).

![logistics](logistics_project.png)

Download the data from the below gdrive and upload into the catalog
https://drive.google.com/drive/folders/1J3AVJIPLP7CzT15yJIpSiWXshu1iLXKn?usp=drive_link

##**1. Data Munging** -

####1. Visibily/Manually opening the file and capture couple of data patterns (Manual Exploratory Data Analysis)

**Logistics source1 file analysis**

- csv comma separated file with header and 5 columns
- Row level duplicates exists
- Null rows and null values exists
- Shipment id & age column has incorrect datatypes
- addtional columns exists

**Logistics source2 file analysis**

- csv comma separated file with header and 7 columns
- Row level duplicates exists
- Null rows and null values exists
- Shipment id & age column has incorrect datatypes
- lesser columns exists

**Logistics shipment detail**

- json file with 11 columns

####2. Programatically try to find couple of data patterns applying below EDA (File: logistics_source1)
1. Apply inferSchema and toDF to create a DF and analyse the actual data.
2. Analyse the schema, datatypes, columns etc.,
3. Analyse the duplicate records count and summary of the dataframe.

In [0]:
from pyspark.sql.session import SparkSession
spark=SparkSession.builder.appName("LogisticsProject").getOrCreate()

rawsource1df=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/logistics/logistics_source1",header=True,inferSchema=True).toDF("shipment_id","firstname","lastname","age","role")

display(rawsource1df)
rawsource1df.printSchema()
rawsource1df.dtypes
rawsource1df.columns

rawsource1df.count()
rawsource1df.distinct().count()
display(rawsource1df.summary())

###a. Passive Data Munging -  (File: logistics_source1  and logistics_source2)
Without modifying the data, identify:<br>
Shipment IDs that appear in both master_v1 and master_v2<br>
Records where:<br>
1. shipment_id is non-numeric
2. age is not an integer<br>

Count rows having:
3. fewer columns than expected
4. more columns than expected

In [0]:
#Create a Spark Session Object
from pyspark.sql.session import SparkSession
spark=SparkSession.builder.appName("LogisticsProject").getOrCreate()

from pyspark.sql.types import StructType,StructField,StringType,IntegerType,ShortType
from pyspark.sql.functions import col,rlike,split,size

rawsource1df=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/logistics/logistics_source1",header=True,inferSchema=True).toDF("shipment_id","first_name","last_name","age","role")
rawsource2df=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/logistics/logistics_source2",header=True,inferSchema=True).toDF("shipment_id","first_name","last_name","age","role","hub_location","vehicle_type")
display(rawsource1df.show(10))
display(rawsource2df.show(10))

#to identify shipment ids present in both source1 & source2 without modifying the data
joinedsourcedf=rawsource1df.join(rawsource2df,how="semi",on="shipment_id")
display(joinedsourcedf.select("shipment_id"))
display(joinedsourcedf.show(10))    

#1. To identify non-numeric shipment_id
nanidsrc1df=rawsource1df.where("shipment_id rlike '[^0-9]'")
display(nanidsrc1df)
nanidsrc2df=rawsource2df.where("shipment_id rlike '[^0-9]'")
display(nanidsrc2df)

#2. To identify age is not an integer
nanagesrc1df=rawsource1df.where("age rlike '[^0-9]'")
display(nanagesrc1df)
nanagesrc2df=rawsource2df.where("age rlike '[^0-9]'")
display(nanagesrc1df)


#3. Rows having fewer column than expected and #4. more columns than expected

#source1
struct1=StructType([StructField('shipment_id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('role', StringType(), True), StructField('corruptedrows',StringType())])

rawsource1corruptdf=spark.read.schema(struct1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/logistics/logistics_source1", header=True, mode = "permissive", columnNameOfCorruptRecord="corruptedrows")
display(rawsource1corruptdf)

actualcolcnt=len(rawsource1df.columns)
display(actualcolcnt)

corruptdf=rawsource1corruptdf.where("corruptedrows is not null")
corruptdflen=corruptdf.withColumn("corruptcolumncnt",size(split(col("corruptedrows"),",")))
display(corruptdflen)

print("one record is having more number of columns than expected in logistics_source1")
print("Two records are having lesser number of columns than expected in logistics_source1")

#source2
struct2=StructType([StructField('shipment_id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('role', StringType(), True), StructField('hub_location', StringType(), True), StructField('vehicle_type', StringType(), True), StructField('corruptedrows',StringType())])

rawsource2corruptdf=spark.read.schema(struct2).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/logistics/logistics_source2", header=True, mode = "permissive", columnNameOfCorruptRecord="corruptedrows")
display(rawsource2corruptdf)

actualcolcnt=len(rawsource2df.columns)
corruptdf1=rawsource2corruptdf.where("corruptedrows is not null")
corruptdf1len=corruptdf1.withColumn("corruptcolumncnt",size(split(col("corruptedrows"),",")))
display(corruptdf1len)

print("Two records are having lesser number of columns than expected in logistics_source2")






###**b. Active Data Munging** File: logistics_source1 and logistics_source2

#####1.Combining Data + Schema Merging (Structuring)
1. Read both files without enforcing schema
2. Align them into a single canonical schema: shipment_id,
first_name,
last_name,
age,
role,
hub_location,
vehicle_type,
data_source
3. Add data_source column with values as: system1, system2 in the respective dataframes

In [0]:
#1. Read both files without enforcing schema

from pyspark.sql.functions import col,lit


struct1="shipment_id integer, firstname string, lastname string, age short, role string"
rawsource1df=spark.read.schema(struct1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/logistics/logistics_source1", header=True)
struct2="shipment_id integer, firstname string, lastname string, age short, role string, hub_location string, vehicle_type string"
rawsource2df=spark.read.schema(struct2).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/logistics/logistics_source2", header=True)
display(rawsource1df.show(10))
display(rawsource2df.show(10))

#3. Add data_source column with values as: system1, system2 in the respective dataframesv 
rawsource1df = rawsource1df.withColumn("data_source", lit("system1"))
rawsource2d = rawsource2df.withColumn("data_source", lit("system2"))

#2. Align them into a single canonical schema: shipment_id, first_name, last_name, age, role, hub_location, vehicle_type, data_source
combinedsourcedf=rawsource1df.unionByName(rawsource2df,allowMissingColumns=True)
display(combinedsourcedf)
combinedsourcedf.printSchema()



#####2. Cleansing, Scrubbing: 
Cleansing (removal of unwanted datasets)<br>
1. Mandatory Column Check - Drop any record where any of the following columns is NULL:shipment_id, role<br>
2. Name Completeness Rule - Drop records where both of the following columns are NULL: first_name, last_name<br>
3. Join Readiness Rule - Drop records where the join key is null: shipment_id<br>

Scrubbing (convert raw to tidy)<br>
4. Age Defaulting Rule - Fill NULL values in the age column with: -1<br>
5. Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN<br>
6. Invalid Age Replacement - Replace the following values in age:
"ten" to -1
"" to -1<br>
7. Vehicle Type Normalization - Replace inconsistent vehicle types: 
truck to LMV
bike to TwoWheeler

In [0]:
#Cleansing (removal of unwanted datasets)
#1. Mandatory Column Check - Drop any record where any of the following columns is NULL:shipment_id, role
combinedsrcdedupdf=combinedsourcedf.na.drop(how='any',subset=["shipment_id","role"])
#display(combinedsrcdedupdf.filter("shipment_id is Null or role is Null"))
display(combinedsrcdedupdf)

#2. Name Completeness Rule - Drop records where both of the following columns are NULL: first_name, last_name
combinedsrcdedup2df=combinedsrcdedupdf.na.drop(how='all',subset=["firstname","lastname"])
#display(combinedsrcdedup2df.filter("firstname is Null and lastname is Null"))

#3. Join Readiness Rule - Drop records where the join key is null: shipment_id
#no shipment_id is having Null
display(combinedsrcdedup2df.filter("shipment_id is Null"))
cleanseddf=combinedsrcdedup2df
display(cleanseddf)

#Scrubbing (convert raw to tidy)

#4. Age Defaulting Rule - Fill NULL values in the age column with: -1
scrubbeddf1=cleanseddf.na.fill(-1,"age")
display(scrubbeddf1.filter("age ==-1")) #to display the records with -1

#5. Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN
scrubbeddf2=scrubbeddf1.na.fill("UNKNOWN","vehicle_type")
display(scrubbeddf2.filter("vehicle_type is NULL")) #no records should be displayed

#6. Invalid Age Replacement - Replace the following values in age: "ten" to -1 "" to -1
scrubbeddf3=scrubbeddf2.na.replace("-1","ten","age")
display(scrubbeddf3.filter("age == -1")) #no records should be displayed

#7. Vehicle Type Normalization - Replace inconsistent vehicle types: truck to LMV bike to TwoWheeler
repl_dict_list={"Truck":"LMV","Bike":"TwoWheeler"}
scrubbeddf4=scrubbeddf3.replace(repl_dict_list,"vehicle_type")
display(scrubbeddf4.filter("vehicle_type in ('Truck','Bike')")) #no records should be displayed

display(scrubbeddf4)



####3. Standardization, De-Duplication and Replacement / Deletion of Data to make it in a usable format

Creating shipments Details data Dataframe creation <br>
1. Create a DF by Reading Data from logistics_shipment_detail.json
2. As this data is a clean json data, it doesn't require any cleansing or scrubbing.

Standardizations:<br>

1. Add a column<br> 
Source File: DF of logistics_shipment_detail_3000.json<br>: domain as 'Logistics',  current timestamp 'ingestion_timestamp' and 'False' as 'is_expedited'
2. Column Uniformity: 
role - Convert to lowercase<br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
vehicle_type - Convert values to UPPERCASE<br>
Source Files: DF of logistics_shipment_detail_3000.json
hub_location - Convert values to initcap case<br>
Source Files: DF of merged(logistics_source1 & logistics_source2)<br>
3. Format Standardization:<br>
Source Files: DF of logistics_shipment_detail_3000.json<br>
Convert shipment_date to yyyy-MM-dd<br>
Ensure shipment_cost has 2 decimal precision<br>
4. Data Type Standardization<br>
Standardizing column data types to fix schema drift and enable mathematical operations.<br>
Source File: DF of merged(logistics_source1 & logistics_source2) <br>
age: Cast String to Integer<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
shipment_weight_kg: Cast to Double<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
is_expedited: Cast to Boolean<br>
5. Naming Standardization <br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
Rename: first_name to staff_first_name<br>
Rename: last_name to staff_last_name<br>
Rename: hub_location to origin_hub_city<br>
6. Reordering columns logically in a better standard format:<br>
Source File: DF of Data from all 3 files<br>
shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)

In [0]:
#standardization
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType,DoubleType
#1. Create a DF by Reading Data from logistics_shipment_detail.json
#2. As this data is a clean json data, it doesn't require any cleansing or scrubbing.

shipdtldf = spark.read.option("multiline", "true").json("/Volumes/workspace/wd36schema/ingestion_volume/source/logistics/logistics_shipment_detail_3000.json")

display(shipdtldf)

#Add a column 
#: domain as 'Logistics', current timestamp 'ingestion_timestamp' and 'False' as 'is_expedited'
shipdtlstd1=shipdtldf.withColumns({"domain":lit("Logistics"),"ingestion_timestamp":current_timestamp(),"is_expedited":lit("False")})
display(shipdtlstd1)

#Column Uniformity: role - Convert to lowercase
#Source File: DF of merged(logistics_source1 & logistics_source2)
#vehicle_type - Convert values to UPPERCASE
#Source Files: DF of logistics_shipment_detail_3000.json hub_location - Convert values to initcap case
#Source Files: DF of merged(logistics_source1 & logistics_source2)

combinedstddf1=scrubbeddf4.withColumn("hub_location",initcap(scrubbeddf4.hub_location)).withColumn("vehicle_type",upper(scrubbeddf4.vehicle_type))
#or
combinedstddf1=scrubbeddf4.withColumn("hub_location",initcap(col("hub_location"))).withColumn("vehicle_type",upper(col("vehicle_type")))
display(combinedstddf1)

#Format Standardization:
#Source Files: DF of logistics_shipment_detail_3000.json
#Convert shipment_date to yyyy-MM-dd
#Ensure shipment_cost has 2 decimal precision

shipdtlstd2=shipdtlstd1.withColumn("shipment_date",to_date(shipdtlstd1.shipment_date,"yyyy-MM-dd")).withColumn("shipment_cost",round(shipdtlstd1.shipment_cost,2))
display(shipdtlstd2)

#Data Type Standardization
#Standardizing column data types to fix schema drift and enable mathematical operations.
#Source File: DF of merged(logistics_source1 & logistics_source2)
#age: Cast String to Integer
combinedstddf2=combinedstddf1.withColumn("age",combinedstddf1.age.cast(IntegerType()))
#combinedstddf2.printSchema()
#display(combinedstddf2)


#Source File: DF of logistics_shipment_detail_3000.json
#shipment_weight_kg: Cast to Double
#Source File: DF of logistics_shipment_detail_3000.json
#is_expedited: Cast to Boolean
shipdtlstd3=shipdtlstd2.withColumn("shipment_weight_kg",shipdtlstd2.shipment_weight_kg.cast("Double")).withColumn("is_expedited",shipdtlstd2.is_expedited.cast("Boolean"))
#shipdtlstd3.printSchema()
#display(shipdtlstd3)

#Naming Standardization
#Source File: DF of merged(logistics_source1 & logistics_source2)
#Rename: first_name to staff_first_name
#Rename: last_name to staff_last_name
#Rename: hub_location to origin_hub_city

combinedstddf3=combinedstddf2.withColumnRenamed("firstname","staff_first_name").withColumnRenamed("lastname","staff_last_name").withColumnRenamed("hub_location","origin_hub_city")
#display(combinedstddf3)

#Reordering columns logically in a better standard format:
#Source File: DF of Data from all 3 files
#shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)

joinedalldf=combinedstddf3.join(shipdtlstd3,how='inner',on='shipment_id').select("shipment_id","staff_first_name","staff_last_name","role","origin_hub_city","shipment_cost","ingestion_timestamp")

display(joinedalldf)

Deduplication:
1. Apply Record Level De-Duplication
2. Apply Column Level De-Duplication (Primary Key Enforcement)

In [0]:
dedupeddf1=joinedalldf.distinct()
dedupeddf1.display()
dedupeddf2=dedupeddf1.dropDuplicates(subset=["shipment_id"])
dedupeddf2.display()

##2. Data Enrichment - Detailing of data
Makes your data rich and detailed <br>

###### Adding of Columns (Data Enrichment)
*Creating new derived attributes to enhance traceability and analytical capability.*

**1. Add Audit Timestamp (`load_dt`)**
Source File: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** We need to track exactly when this record was ingested into our Data Lakehouse for auditing purposes.
* **Action:** Add a column `load_dt` using the function `current_timestamp()`.

**2. Create Full Name (`full_name`)**
Source File: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** The reporting dashboard requires a single field for the driver's name instead of separate columns.
* **Action:** Create `full_name` by concatenating `first_name` and `last_name` with a space separator.
* **Result:** "Rajesh" + " " + "Kumar" -> **"Rajesh Kumar"**

**3. Define Route Segment (`route_segment`)**
Source File: DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** The logistics team wants to analyze performance based on specific transport lanes (Source to Destination).
* **Action:** Combine `source_city` and `destination_city` with a hyphen.
* **Result:** "Chennai" + "-" + "Pune" -> **"Chennai-Pune"**

**4. Generate Vehicle Identifier (`vehicle_identifier`)**
Source File: DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** We need a unique tracking code that immediately tells us the vehicle type and the shipment ID.
* **Action:** Combine `vehicle_type` and `shipment_id` to create a composite key.
* **Result:** "Truck" + "_" + "500001" -> **"Truck_500001"**

###### Deriving of Columns (Time Intelligence)
*Extracting temporal features from dates to enable period-based analysis and reporting.*<br>
Source File: logistics_shipment_detail_3000.json<br>
**1. Derive Shipment Year (`shipment_year`)**
* **Scenario:** Management needs an annual performance report to compare growth year-over-year.
* **Action:** Extract the year component from `shipment_date`.
* **Result:** "2024-04-23" -> **2024**

**2. Derive Shipment Month (`shipment_month`)**
* **Scenario:** Analysts want to identify seasonal peaks (e.g., increased volume in December).
* **Action:** Extract the month component from `shipment_date`.
* **Result:** "2024-04-23" -> **4** (April)

**3. Flag Weekend Operations (`is_weekend`)**
* **Scenario:** The Operations team needs to track shipments handled during weekends to calculate overtime pay or analyze non-business day capacity.
* **Action:** Flag as **'True'** if the `shipment_date` falls on a Saturday or Sunday.

**4. Flag shipment status (`is_expedited`)**
* **Scenario:** The Operations team needs to track shipments is IN_TRANSIT or DELIVERED.
* **Action:** Flag as **'True'** if the `shipment_status` IN_TRANSIT or DELIVERED.

###### Enrichment/Business Logics (Calculated Fields)
*Deriving new metrics and financial indicators using mathematical and date-based operations.*<br>
Source File: logistics_shipment_detail_3000.json<br>

**1. Calculate Unit Cost (`cost_per_kg`)**
* **Scenario:** The Finance team wants to analyze the efficiency of shipments by determining the cost incurred per unit of weight.
* **Action:** Divide `shipment_cost` by `shipment_weight_kg`.
* **Logic:** `shipment_cost / shipment_weight_kg`

**2. Track Shipment Age (`days_since_shipment`)**
* **Scenario:** The Operations team needs to monitor how long it has been since a shipment was dispatched to identify potential delays.
* **Action:** Calculate the difference in days between the `current_date` and the `shipment_date`.
* **Logic:** `datediff(current_date(), shipment_date)`

**3. Compute Tax Liability (`tax_amount`)**
* **Scenario:** For invoicing and compliance, we must calculate the Goods and Services Tax (GST) applicable to each shipment.
* **Action:** Calculate 18% GST on the total `shipment_cost`.
* **Logic:** `shipment_cost * 0.18`

In [0]:
#Data Enrichment 

## adding of Columns ##
#1. Add Audit Timestamp (load_dt) 
enricheddf=combinedsourcedf.withColumn("load_dt",current_timestamp())
#2. Create Full Name
enricheddf1=enricheddf.select("shipment_id",concat("firstname",lit(" "),"lastname").alias("full_name"),"firstname","lastname","age","role","data_source","hub_location","vehicle_type","load_dt")

#3. Define Route Segment & 4.Generate Vehicle Identifier
enrichshipdtldf=shipdtlstd3.withColumn("route_segment",concat("source_city",lit("-"),"destination_city")).withColumn("vehicle_identifier",concat("vehicle_type",lit("_"),"shipment_id"))

#select("cargo_type",concat("source_city",lit"-","destination_city").alias("route_segment"),"destination_city","order_id","payment_mode","shipment_cost","shipment_date","shipment_id","shipment_status","shipment_weight_kg","source_city","vehicle_type","domain","ingestion_timestamp","is_expedited")

## Deriving of columns ##
#1.Derive Shipment Year & 2. Derive Shipment Month
enrichshipdtldf1=enrichshipdtldf.withColumn("shipment_year",year(col("shipment_date"))).withColumn("shipment_month",month(col("shipment_date")))

#3. Flag Weekend Operations
# dayofweek: 1=Sunday, 7=Saturday
enrichshipdtldf2=enrichshipdtldf1.withColumn("is_weekend",when(dayofweek(col("shipment_date")).isin([1,7]), "True").otherwise("False"))

#4. Flag shipment status 
enrichshipdtldf2.withColumn("is_expedited",when(col("shipment_status").isin(["IN_TRANSIT","DELIVERED"]), "True").otherwise("False")).display()

## Enrichment/Business Logics (Calculated Fields) ##

#1. Calculate Unit Cost (cost_per_kg) & 2. Track Shipment Age (days_since_shipment) & 3. Compute Tax Liability (tax_amount)

enrichshipdtldf2.withColumn("cost_per_kg",col("shipment_cost")/col("shipment_weight_kg")).withColumn("days_since_shipment",datediff("current_date","shipment_date")).withColumn("tax_amount",col("shipment_cost")*0.18).display()


enrichshipdtldf2.display()
enricheddf1.display()

###### Remove/Eliminate (drop, select, selectExpr)
*Excluding unnecessary or redundant columns to optimize storage and privacy.*<br>
Source File: DF of logistics_source1 and logistics_source2<br>

**1. Remove Redundant Name Columns**
* **Scenario:** Since we have already created the `full_name` column in the Enrichment step, the individual name columns are now redundant and clutter the dataset.
* **Action:** Drop the `first_name` and `last_name` columns.
* **Logic:** `df.drop("first_name", "last_name")`

In [0]:
enricheddf2=enricheddf1.drop("firstname","lastname")
#or
enricheddf2=enricheddf1.select("shipment_id","full_name","age","role","data_source","hub_location","vehicle_type","load_dt")
#or
enricheddf2=enricheddf1.selectExpr("shipment_id","full_name","age","role","data_source","hub_location","vehicle_type","load_dt")
enricheddf2.display()


##### Splitting & Merging/Melting of Columns
*Reshaping columns to extract hidden values or combine fields for better analysis.*<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
**1. Splitting (Extraction)**
*Breaking one column into multiple to isolate key information.*
* **Split Order Code:**
  * **Action:** Split `order_id` ("ORD100000") into two new columns:
    * `order_prefix` ("ORD")
    * `order_sequence` ("100000")
* **Split Date:**
  * **Action:** Split `shipment_date` into three separate columns for partitioning:
    * `ship_year` (2024)
    * `ship_month` (4)
    * `ship_day` (23)

**2. Merging (Concatenation)**
*Combining multiple columns into a single unique identifier or description.*
* **Create Route ID:**
  * **Action:** Merge `source_city` ("Chennai") and `destination_city` ("Pune") to create a descriptive route key:
    * `route_lane` ("Chennai->Pune")

In [0]:

enrichshipdtldf3=enrichshipdtldf2.withColumn("order_prefix",substring("order_id",1,3)).withColumn("order_sequence",substring("order_id",4,6))\
    .withColumn("ship_year",year("shipment_date")).withColumn("ship_month",month("shipment_date")).withColumn("ship_day",day("shipment_date"))\
        .withColumn("route_lane",concat("source_city",lit("->"),"destination_city"))
enrichshipdtldf3.display()

## 3. Data Customization & Processing - Application of Tailored Business Specific Rules

### **UDF1: Complex Incentive Calculation**
**Scenario:** The Logistics Head wants to calculate a "Performance Bonus" for drivers based on tenure and role complexity.

**Action:** Create a Python function `calculate_bonus(role, age)` and register it as a Spark UDF.

**Logic:**
* **IF** `Role` == 'Driver' **AND** `Age` > 50:
  * `Bonus` = 15% of Salary (Reward for Seniority)
* **IF** `Role` == 'Driver' **AND** `Age` < 30:
  * `Bonus` = 5% of Salary (Encouragement for Juniors)
* **ELSE**:
  * `Bonus` = 0

**Result:** A new derived column `projected_bonus` is generated for every row in the dataset.

---

### **UDF2: PII Masking (Privacy Compliance)**
**Scenario:** For the analytics dashboard, we must hide the full identity of the staff to comply with privacy laws (GDPR/DPDP), while keeping names recognizable for internal managers.

**Business Rule:** Show the first 2 letters, mask the middle characters with `****`, and show the last letter.

**Action:** Create a UDF `mask_identity(name)`.

**Example:**
* **Input:** `"Rajesh"`
* **Output:** `"Ra****h"`
<br>
**Note: Convert the above udf logic to inbult function based transformation to ensure the performance is improved.**

In [0]:
def calculate_bonus(role, age):
  if role == 'Driver' and age > 50:
    return 0.15
  elif role == 'Driver' and age < 30:
    return 0.05
  else:
    return 0

def mask_identity(name):
  if name == None:
    return None
  else:
    return name[0:2]+"****"+name[-1]
  
#mask_identity("John")  
#print(calculate_bonus("Driver",25))
from pyspark.sql.functions import udf
udfrole=udf(calculate_bonus)
udfmask=udf(mask_identity)
enricheddf3=enricheddf2.withColumn("projected_bonus",udfrole(enricheddf2.role,enricheddf2.age)).withColumn("full_name",udfmask(enricheddf2.full_name))
enricheddf3.display()




## 4. Data Core Curation & Processing (Pre-Wrangling)
*Applying business logic to focus, filter, and summarize data before final analysis.*

**1. Select (Projection)**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** The Driver App team only needs location data, not sensitive HR info.
* **Action:** Select only `first_name`, `role`, and `hub_location`.

**2. Filter (Selection)**<br>
Source File: DF of json<br>
* **Scenario:** We need a report on active operational problems.
* **Action:** Filter rows where `shipment_status` is **'DELAYED'** or **'RETURNED'**.
* **Scenario:** Insurance audit for senior staff.
* **Action:** Filter rows where `age > 50`.

**3. Derive Flags & Columns (Business Logic)**<br>
Source File: DF of json<br>
* **Scenario:** Identify high-value shipments for security tracking.
* **Action:** Create flag `is_high_value` = **True** if `shipment_cost > 50,000`.
* **Scenario:** Flag weekend operations for overtime calculation.
* **Action:** Create flag `is_weekend` = **True** if day is Saturday or Sunday.

**4. Format (Standardization)**<br>
Source File: DF of json<br>
* **Scenario:** Finance requires readable currency formats.
* **Action:** Format `shipment_cost` to string like **"₹30,695.80"**.
* **Scenario:** Standardize city names for reporting.
* **Action:** Format `source_city` to Uppercase (e.g., "chennai" → **"CHENNAI"**).

**5. Group & Aggregate (Summarization)**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** Regional staffing analysis.
* **Action:** Group by `hub_location` and **Count** the number of staff.
* **Scenario:** Fleet capacity analysis.
* **Action:** Group by `vehicle_type` and **Sum** the `shipment_weight_kg`.

**6. Sorting (Ordering)**<br>
Source File: DF of json<br>
* **Scenario:** Prioritize the most expensive shipments.
* **Action:** Sort by `shipment_cost` in **Descending** order.
* **Scenario:** Organize daily dispatch schedule.
* **Action:** Sort by `shipment_date` (Ascending) then `priority_flag` (Descending).

**7. Limit (Top-N Analysis)**<br>
Source File: DF of json<br>
* **Scenario:** Dashboard snapshot of critical delays.
* **Action:** Filter for 'DELAYED', Sort by Cost, and **Limit to top 10** rows.

In [0]:
#1. Select (Projection)
curateddf1=enricheddf3.select("full_name", "role", "hub_location")

#2. Filter (Selection)
curatedshipdtldf1=enrichshipdtldf3.filter((enrichshipdtldf3.shipment_status =="DELAYED") | (enrichshipdtldf3.shipment_status =="RETURNED"))

curateddf2=enricheddf3.select("*").filter(enricheddf3.age > 50)
curateddf2.display()

#3. Derive Flags & Columns (Business Logic) & #4. Format (Standardization)
curatedshipdtldf2=curatedshipdtldf1.withColumn("is_high_value",when(col("shipment_cost") > 50000,True).otherwise(False))\
    .withColumn("is_weekend",when(dayofweek(col("shipment_date")).isin([1,7]), "True").otherwise("False"))\
        .withColumn("shipment_cost",concat(lit("₹"),format_number(col("shipment_cost"),2)))\
            .withColumn("source_city",upper(col("source_city")))

#5. Group & Aggregate (Summarization)
curateddf3=curateddf2.groupBy("hub_location").count()

curatedshipdtldf3=curatedshipdtldf2.groupBy(curatedshipdtldf2.vehicle_type).agg(sum("shipment_weight_kg"))

#**6. Sorting (Ordering)**<br>
curatedshipdtldf4=curatedshipdtldf2.orderBy("shipment_cost", ascending=False)
curatedshipdtldf5=curatedshipdtldf2.orderBy("shipment_date","shipment_cost",ascending=[True,False])

curatedshipdtldf5.display()

#**7. Limit (Top-N Analysis)**<br>
curatedshipdtldf6=curatedshipdtldf5.filter("shipment_status == 'DELAYED'").orderBy("shipment_cost", ascending=False).limit(10)



## 5. Data Wrangling - Transformation & Analytics
*Combining, modeling, and analyzing data to answer complex business questions.*

### **1. Joins**
Source Files:<br>
Left Side (staff_df):<br> DF of logistics_source1 & logistics_source2<br>
Right Side (shipments_df):<br> DF of logistics_shipment_detail_3000.json<br>
#### **1.1 Frequently Used Simple Joins (Inner, Left)**
* **Inner Join (Performance Analysis):**
  * **Scenario:** We only want to analyze *completed work*. Connect Staff to the Shipments they handled.
  * **Action:** Join `staff_df` and `shipments_df` on `shipment_id`.
  * **Result:** Returns only rows where a staff member is assigned to a valid shipment.
* **Left Join (Idle Resource check):**
  * **Scenario:** Find out which staff members are currently *idle* (not assigned to any shipment).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right) on `shipment_id`. Filter where `shipments_df.shipment_id` is NULL.

#### **1.2 Infrequent Simple Joins (Self, Right, Full, Cartesian)**
* **Self Join (Peer Finding):**
  * **Scenario:** Find all pairs of employees working in the same `hub_location`.
  * **Action:** Join `staff_df` to itself on `hub_location`, filtering where `staff_id_A != staff_id_B`.
* **Right Join (Orphan Data Check):**
  * **Scenario:** Identify shipments in the system that have *no valid driver* assigned (Data Integrity Issue).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right). Focus on NULLs on the left side.
* **Full Outer Join (Reconciliation):**
  * **Scenario:** A complete audit to find *both* idle drivers AND unassigned shipments in one view.
  * **Action:** Perform a Full Outer Join on `shipment_id`.
* **Cartesian/Cross Join (Capacity Planning):**
  * **Scenario:** Generate a schedule of *every possible* driver assignment to *every* pending shipment to run an optimization algorithm.
  * **Action:** Cross Join `drivers_df` and `pending_shipments_df`.

#### **1.3 Advanced Joins (Semi and Anti)**
* **Left Semi Join (Existence Check):**
  * **Scenario:** "Show me the details of Drivers who have *at least one* shipment." (Standard filtering).
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_semi")`.
  * **Benefit:** Performance optimization; it stops scanning the right table once a match is found.
* **Left Anti Join (Negation Check):**
  * **Scenario:** "Show me the details of Drivers who have *never* touched a shipment."
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_anti")`.

### **2. Lookup**<br>
Source File: DF of logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Validation. Check if the `hub_location` in the staff file exists in the corporate `Master_City_List`.
* **Action:** Compare values against a reference list.

### **3. Lookup & Enrichment**<br>
Source File: DF of logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Geo-Tagging.
* **Action:** Lookup `hub_location` ("Pune") in a Master Latitude/Longitude table and enrich the dataset by adding `lat` and `long` columns for map plotting.

### **4. Schema Modeling (Denormalization)**<br>
Source Files: DF of All 3 Files (logistics_source1, logistics_source2, logistics_shipment_detail_3000.json)<br>
* **Scenario:** Creating a "Gold Layer" Table for PowerBI/Tableau.
* **Action:** Flatten the Star Schema. Join `Staff`, `Shipments`, and `Vehicle_Master` into one wide table (`wide_shipment_history`) so analysts don't have to perform joins during reporting.

### **5. Windowing (Ranking & Trends)**<br>
Source Files:<br>
DF of logistics_source2: Provides hub_location (Partition Key).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Ordering Key)<br>
* **Scenario:** "Who are the Top 3 Drivers by Cost in *each* Hub?"
* **Action:**
  1. Partition by `hub_location`.
  2. Order by `total_shipment_cost` Descending.
  3. Apply `dense_rank()` and `row_number()
  4. Filter where `rank or row_number <= 3`.

### **6. Analytical Functions (Lead/Lag)**<br>
Source File: <br>
DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** Idle Time Analysis.
* **Action:** For each driver, calculate the days elapsed since their *previous* shipment.

### **7. Set Operations**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Union:** Combining `Source1` (Legacy) and `Source2` (Modern) into one dataset (Already done in Active Munging).
* **Intersect:** Identifying Staff IDs that appear in *both* Source 1 and Source 2 (Duplicate/Migration Check).
* **Except (Difference):** Identifying Staff IDs present in Source 2 but *missing* from Source 1 (New Hires).

### **8. Grouping & Aggregations (Advanced)**<br>
Source Files:<br>
DF of logistics_source2: Provides hub_location and vehicle_type (Grouping Dimensions).<br>
DF of logistics_shipment_detail_3000.json: Provides shipment_cost (Aggregation Metric).<br>
* **Scenario:** The CFO wants a subtotal report at multiple levels:
  1. Total Cost by Hub.
  2. Total Cost by Hub AND Vehicle Type.
  3. Grand Total.
* **Action:** Use `cube("hub_location", "vehicle_type")` or `rollup()` to generate all these subtotals in a single query.

##6. Data Persistance (LOAD)-> Data Publishing & Consumption<br>

Store the inner joined, lookup and enrichment, Schema Modeling, windowing, analytical functions, set operations, grouping and aggregation data into the delta tables.

##7.Take the copy of the above notebook and try to write the equivalent SQL for which ever applicable.